<a href="https://colab.research.google.com/github/adalbertii/LLM/blob/main/spelling_correction_t5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 whi

In [ ]:
import torch
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer,
)
from datasets import load_dataset

In [ ]:
dataset = load_dataset(
    'text',
    data_files='wikipedia.txt',
    split='train',
)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 1922
})


In [ ]:
print(dataset[0])

{'text': 'Apennines: Apenines Appenines'}


In [ ]:
dataset_full = dataset.train_test_split(shuffle=True, test_size=0.2)


In [ ]:
dataset_train, dataset_valid = dataset_full['train'], dataset_full['test']
print(dataset_train)
print(dataset_valid)

Dataset({
    features: ['text'],
    num_rows: 1537
})
Dataset({
    features: ['text'],
    num_rows: 385
})


In [ ]:
model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
dataset_train['text'][:5]

['attitude: attitide',
 'then: thne',
 'adaptation: adaption',
 'sepulcher: sepulchure sepulcre',
 'one: noone']

In [ ]:
all_correct = [word.split(': ')[0] for word in dataset_train['text']]

In [ ]:
all_correct[:5]

['attitude', 'then', 'adaptation', 'sepulcher', 'one']

In [ ]:
all_wrong = [f"fix_spelling: {word.split(': ')[1].split(' ')[0]}" for word in dataset_train['text']]

In [ ]:
all_wrong[:5]

['fix_spelling: attitide',
 'fix_spelling: thne',
 'fix_spelling: adaption',
 'fix_spelling: sepulchure',
 'fix_spelling: noone']

In [ ]:
# Function to convert text data into model inputs and targets
def preprocess_function(examples):
    all_correct = [word.split(': ')[0] for word in examples['text']]
    all_wrong = [f"fix_spelling: {word.split(': ')[1].split(' ')[0]}" for word in examples['text']]
    # print('CORRECT')
    # print(all_correct)
    # print('WRONG')
    # print(all_wrong)
    model_inputs = tokenizer(
        all_wrong,
        max_length=32,
        truncation=True,
        padding='max_length'
    )
    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            all_correct,
            max_length=32,
            truncation=True,
            padding='max_length'
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# Apply the function to the whole dataset
tokenized_train = dataset_train.map(
    preprocess_function,
    batched=True,
    num_proc=8
)
tokenized_valid = dataset_valid.map(
    preprocess_function,
    batched=True,
    num_proc=8
)

Map (num_proc=8):   0%|          | 0/1537 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

Map (num_proc=8):   0%|          | 0/385 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

In [ ]:
# Load the pre-trained model.
model = T5ForConditionalGeneration.from_pretrained(model_name)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Specify the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
model.to(device)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")


60,506,624 total parameters.


In [ ]:
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

60,506,624 training parameters.


In [ ]:
# Define the training arguments
out_dir = 'results_t5'
batch_size = 32
epochs = 100
training_args = TrainingArguments(
    output_dir=out_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=out_dir,
    logging_steps=10,
    evaluation_strategy='steps',
    save_steps=100,
    eval_steps=100,
    load_best_model_at_end=True,
    save_total_limit=10,
    report_to='tensorboard',
    learning_rate=0.00005,
    dataloader_num_workers=8,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
)


In [ ]:
history = trainer.train()

In [ ]:
tokenizer.save_pretrained(out_dir)

# Inference using the Trained Spelling Correction Model

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_path = f"results_t5/checkpoint-4900/"  # the path where you saved your model
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained('results_t5')

In [ ]:
def do_correction(text, model, tokenizer):
    input_text = f"fix_spelling: {text}"
    inputs = tokenizer.encode(
        input_text,
        return_tensors='pt',
        max_length=32,
        padding='max_length',
        truncation=True
    )
    # Get correct sentence ids.
    corrected_ids = model.generate(
        inputs,
        max_length=32,
        num_beams=5, # `num_beams=1` indicated temperature sampling.
        early_stopping=True
    )
    # Decode.
    corrected_sentence = tokenizer.decode(
        corrected_ids[0],
        skip_special_tokens=True
    )
    return corrected_sentence

In [ ]:
sentences = [
    'mysogynist',
    'boundare',
    'abondoned',
    'transcripting'
]


In [ ]:
for sentence in sentences:
    corrected_sentence = do_correction(sentence, model, tokenizer)
    print(f"ORIG: {sentence}\nPRED: {corrected_sentence}\n")